In [40]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statistics as st

def get_df(day):
    file_name = f"prices_round_3_day_{day}.csv"
    return pd.read_csv(file_name, sep=';')

def get_product(df, product):
    return df[df['product'] == product].copy()

def get_first_three_dfs():
    first_df = get_df(0)
    second_df = get_df(1)
    third_df = get_df(2)

    second_df['timestamp'] = second_df['timestamp'] + 1000000
    third_df['timestamp'] += 2_000_000
    return pd.concat([first_df, second_df, third_df])



In [41]:
df = get_first_three_dfs()

In [42]:

df_volcanic_rock = get_product(df, 'VOLCANIC_ROCK')
df_volcanic_rock_voucher_10000 = get_product(df, "VOLCANIC_ROCK_VOUCHER_10000")
df_volcanic_rock_voucher_10000 = df_volcanic_rock_voucher_10000.merge(df_volcanic_rock[['timestamp', 'mid_price']], on='timestamp', suffixes=('', '_VOLCANIC_ROCK'))


In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_volcanic_rock['timestamp'], y=df_volcanic_rock['mid_price'], name='Volcanic Rock Mid Price'))
fig.add_trace(go.Scatter(x=df_volcanic_rock_voucher_10000['timestamp'], y=df_volcanic_rock_voucher_10000['mid_price'], name='Volcanic Rock Voucher Mid Price', yaxis='y2'))
fig.update_layout(title='Volcanic Rock and Volcanic Rock Voucher 10000 Prices over Timestamp', yaxis=dict(title='Volcanic Rock Mid Price'), yaxis2=dict(title='Volcanic Rock Voucher 10000 Bid Price', overlaying='y', side='right'))
fig.show()


In [44]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_call(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    call_price = (spot * norm.cdf(d1) - strike * norm.cdf(d2))
    return call_price

def black_scholes_put(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot / strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    put_price = (strike * norm.cdf(-d2) - spot * norm.cdf(-d1))
    return put_price

def delta(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.cdf(d1)

def gamma(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1)/(spot * volatility * np.sqrt(time_to_expiry))

def vega(spot, strike, time_to_expiry, volatility):
    d1 = (np.log(spot) - np.log(strike) + (0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    return norm.pdf(d1) * (spot * np.sqrt(time_to_expiry)) / 100

def implied_volatility(call_price, spot, strike, time_to_expiry):
    # Define the equation where the root is the implied volatility
    def equation(volatility):
        estimated_price = black_scholes_call(spot, strike, time_to_expiry, volatility)
        return estimated_price - call_price

    # Using Brent's method to find the root of the equation
    implied_vol = brentq(equation, 1e-10, 3.0, xtol=1e-10)
    return implied_vol

def realized_vol(df_volcanic_rock_voucher_10000, window, step_size):
    df_volcanic_rock_voucher_10000[f'log_return_{step_size}'] = np.log(df_volcanic_rock_voucher_10000['mid_price_VOLCANIC_ROCK'].to_numpy()/df_volcanic_rock_voucher_10000['mid_price_VOLCANIC_ROCK'].shift(step_size).to_numpy())
    dt = step_size / 250 / 10000 
    df_volcanic_rock_voucher_10000[f'realized_vol_{step_size}'] = df_volcanic_rock_voucher_10000[f'log_return_{step_size}'].rolling(window=window).apply(lambda x: np.mean(x[::step_size]**2) / dt)
    df_volcanic_rock_voucher_10000[f'realized_vol_{step_size}'] = np.sqrt(df_volcanic_rock_voucher_10000[f'realized_vol_{step_size}'].to_numpy())
    return df_volcanic_rock_voucher_10000



In [45]:
spot_price = 10218        # Spot price of the underlying asset
strike_price = 10000   # Strike price of the option
call_price = 233         # Market price of the call option
time_to_expiry = 5      # Time to expiry in years
initial_guess = 16
#df_volcanic_rock_voucher_10000['implied_vol'] = df_volcanic_rock_voucher_10000.apply(lambda row: implied_volatility(row['mid_price'], row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry), axis=1)
def safe_implied_vol(row):
    try:
        return implied_volatility(row['mid_price'], row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry)
    except Exception as e:
        # Optional: print or log the error
        # print(f"Failed on row {row.name} with error: {e}")
        return np.nan  # or 0, or a fallback value

df_volcanic_rock_voucher_10000['implied_vol'] = df_volcanic_rock_voucher_10000.apply(safe_implied_vol, axis=1)
df_volcanic_rock_voucher_10000['delta'] = df_volcanic_rock_voucher_10000.apply(lambda row: delta(row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_volcanic_rock_voucher_10000['gamma'] = df_volcanic_rock_voucher_10000.apply(lambda row: gamma(row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_volcanic_rock_voucher_10000['vega'] = df_volcanic_rock_voucher_10000.apply(lambda row: vega(row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry, row['implied_vol']), axis=1)

In [46]:
print(df_volcanic_rock_voucher_10000.columns.tolist())


['day', 'timestamp', 'product', 'bid_price_1', 'bid_volume_1', 'bid_price_2', 'bid_volume_2', 'bid_price_3', 'bid_volume_3', 'ask_price_1', 'ask_volume_1', 'ask_price_2', 'ask_volume_2', 'ask_price_3', 'ask_volume_3', 'mid_price', 'profit_and_loss', 'mid_price_VOLCANIC_ROCK', 'implied_vol', 'delta', 'gamma', 'vega']


In [47]:
import plotly.express as px

fig = px.line(df_volcanic_rock_voucher_10000, x='timestamp', y='delta', title='Delta over Time')
fig.show()


In [48]:
fig = px.line(df_volcanic_rock_voucher_10000, x='timestamp', y='vega', title='Vega over Time')
fig.show()

In [49]:
fig = px.line(df_volcanic_rock_voucher_10000, x='timestamp', y='implied_vol', title='Implied Volatility over Time')
fig.show()


# backtest strat-trading implied volatility to mean

In [50]:
day = 1

In [51]:
df = get_df(day)

In [52]:
strike_price = 9500  
time_to_expiry = 5/252
initial_guess = 16

df_volcanic_rock = get_product(df, 'VOLCANIC_ROCK')
df_volcanic_rock_voucher_9500 = get_product(df, "VOLCANIC_ROCK_VOUCHER_9500")
df_volcanic_rock_voucher_9500 = df_volcanic_rock_voucher_9500.merge(df_volcanic_rock[['timestamp', 'mid_price']], on='timestamp', suffixes=('', '_volcanic_rock'))


#df_volcanic_rock_voucher_9500['implied_vol'] = df_volcanic_rock_voucher_9500.apply(lambda row: implied_volatility(row['mid_price'], row['mid_price_volcanic_rock'], strike_price, time_to_expiry), axis=1)
def safe_implied_vol(row):
    try:
        return implied_volatility(row['mid_price'], row['mid_price_VOLCANIC_ROCK'], strike_price, time_to_expiry)
    except Exception as e:
        # Optional: print or log the error
        # print(f"Failed on row {row.name} with error: {e}")
        return np.nan  # or 0, or a fallback value
    
df_volcanic_rock_voucher_9500['delta'] = df_volcanic_rock_voucher_9500.apply(lambda row: delta(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_volcanic_rock_voucher_9500['gamma'] = df_volcanic_rock_voucher_9500.apply(lambda row: gamma(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)
df_volcanic_rock_voucher_9500['vega'] = df_volcanic_rock_voucher_9500.apply(lambda row: vega(row['mid_price_volcanic_rock'], strike_price, time_to_expiry, row['implied_vol']), axis=1)

KeyError: 'implied_vol'

In [ ]:
print(df_volcanic_rock_voucher_9500.columns.tolist())

['day', 'timestamp', 'product', 'bid_price_1', 'bid_volume_1', 'bid_price_2', 'bid_volume_2', 'bid_price_3', 'bid_volume_3', 'ask_price_1', 'ask_volume_1', 'ask_price_2', 'ask_volume_2', 'ask_price_3', 'ask_volume_3', 'mid_price', 'profit_and_loss', 'mid_price_volcanic_rock']


In [ ]:
df_backtest = df_volcanic_rock_voucher_9500[['timestamp', 'mid_price', 'mid_price_volcanic_rock', 'implied_vol', 'delta', 'vega']]
df_backtest = df_backtest.rename(columns={'mid_price': 'mid_price_coupon'})

KeyError: "['implied_vol', 'delta', 'vega'] not in index"

In [ ]:
df_backtest['implied_vol'].mean()

NameError: name 'df_backtest' is not defined

In [ ]:
implied_vol_mean = 0.16

In [ ]:
import pandas as pd

# Set the threshold values
upper_threshold = 0.006  # Threshold for selling option
lower_threshold = -0.006  # Threshold for buying option
close_threshold = 0.0001  # Threshold for clearing position

# Initialize variables
position = 0
pnl = 0
vega_pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for idx, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    if idx == 0:
        continue
    prev_implied_vol = df_backtest.iloc[idx-1]['implied_vol']
    mid_price_coupon = row['mid_price_coupon']
    mid_price_coconut = row['mid_price_coconut']
    vega = row['vega']
    d = row['delta']

    # Check if implied vol is above the upper threshold and no current position
    if implied_vol > implied_vol_mean + upper_threshold and position == 0:
        # Sell 1 delta hedged option
        position = -1
        entry_price_coupon = mid_price_coupon
        entry_price_coconut = mid_price_coconut
        trade_history.append((-1, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is below the lower threshold and no current position
    elif implied_vol < implied_vol_mean + lower_threshold and position == 0:
        # Buy 1 delta hedged option
        position = 1
        entry_price_coupon = mid_price_coupon
        entry_price_coconut = mid_price_coconut
        trade_history.append((1, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is within the close threshold and there is a current position
    elif abs(implied_vol - implied_vol_mean) <= close_threshold and position != 0:
        # Clear the position
        pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))
        position = 0
        trade_history.append((0, mid_price_coupon, mid_price_coconut, implied_vol))

    if position != 0:
        vega_pnl += position * vega * (implied_vol - prev_implied_vol) * 100
# Calculate final PnL if there is still an open position
if position != 0:
    pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Position: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")

Trade History:
Position: 1, Option Price: 586.5, Underlying Price: 9948.0, Implied Volatility: 0.15399524752672736
Position: 0, Option Price: 613.5, Underlying Price: 9954.5, Implied Volatility: 0.15995343437950454
Position: 1, Option Price: 598.5, Underlying Price: 9973.5, Implied Volatility: 0.15367467401239673
Position: 0, Option Price: 618.5, Underlying Price: 9963.5, Implied Volatility: 0.16003033333350636
Position: -1, Option Price: 638.5, Underlying Price: 9956.0, Implied Volatility: 0.16606009972495858
Position: 0, Option Price: 661.5, Underlying Price: 10044.0, Implied Volatility: 0.1600576652344406
Position: -1, Option Price: 676.5, Underlying Price: 10026.5, Implied Volatility: 0.16620222541361077
Position: 0, Option Price: 667.5, Underlying Price: 10055.0, Implied Volatility: 0.16006128368927394
Position: 1, Option Price: 646.5, Underlying Price: 10061.0, Implied Volatility: 0.15397101500535001
Position: 0, Option Price: 663.5, Underlying Price: 10047.5, Implied Volatility:

In [ ]:
vega_pnl * 600

87004.73740732671

In [ ]:
import pandas as pd

# Set the threshold values
upper_threshold = 0.005  # Threshold for selling option
lower_threshold = -0.005  # Threshold for buying option

# Initialize variables
position = 0
pnl = 0
trade_history = []

# Iterate over each row in the dataframe
for _, row in df_backtest.iterrows():
    implied_vol = row['implied_vol']
    mid_price_coupon = row['mid_price_coupon']
    mid_price_coconut = row['mid_price_coconut']
    d = row['delta']

    # Check if implied vol is above the upper threshold
    if implied_vol > implied_vol_mean + upper_threshold:
        # Sell to target position of -1
        if position > -1:
            quantity = -1 - position
            position = -1
            entry_price_coupon = mid_price_coupon
            entry_price_coconut = mid_price_coconut
            trade_history.append((quantity, entry_price_coupon, entry_price_coconut, implied_vol))

    # Check if implied vol is below the lower threshold
    elif implied_vol < implied_vol_mean + lower_threshold:
        # Buy to target position of 1
        if position < 1:
            quantity = 1 - position
            position = 1
            entry_price_coupon = mid_price_coupon
            entry_price_coconut = mid_price_coconut
            trade_history.append((quantity, entry_price_coupon, entry_price_coconut, implied_vol))

# Calculate final PnL for the remaining position
if position != 0:
    pnl += position * (mid_price_coupon - entry_price_coupon + d * (entry_price_coconut - mid_price_coconut))

# Print the trade history and final PnL
print("Trade History:")
for trade in trade_history:
    print(f"Quantity: {trade[0]}, Option Price: {trade[1]}, Underlying Price: {trade[2]}, Implied Volatility: {trade[3]}")

print(f"\nFinal PnL: {pnl}")

Trade History:
Quantity: 1, Option Price: 588.5, Underlying Price: 9944.5, Implied Volatility: 0.15495607230821157
Quantity: -2, Option Price: 639.0, Underlying Price: 9964.5, Implied Volatility: 0.1650645587541441
Quantity: 2, Option Price: 650.5, Underlying Price: 10062.0, Implied Volatility: 0.154837279111737

Final PnL: -9.25964891012682
